# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
%matplotlib 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

from api_keys import g_key

Using matplotlib backend: Qt5Agg


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../WeatherPy/output_data/weather_city_data.csv"

data_weather = pd.read_csv(csv_path)
data_weather.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Yankton,20,US,1603268376,93,42.87,-97.40,37.40,5.82
1,Mar del Plata,0,AR,1603267933,93,-38.00,-57.56,52.00,6.46
2,Albany,97,US,1603268170,86,42.60,-73.97,48.99,1.36
3,Port Blair,36,IN,1603268313,75,11.67,92.75,84.40,14.58
4,Punta Arenas,75,CL,1603267925,74,-53.15,-70.92,37.40,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = data_weather["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = data_weather[["Lat", "Lng"]]

fig = gmaps.figure()
heat_humidity_layer = gmaps.heatmap_layer(locations, 
                                          weights=humidity, 
                                          dissipating=False,
                                          max_intensity=maxhumidity, 
                                          point_radius=3)
fig.add_layer(heat_humidity_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = data_weather.loc[(data_weather["Wind Speed"] <= 10) & 
                                         (data_weather["Cloudiness"] == 0) & \
                                         (data_weather["Max Temp"] >= 70) & 
                                         (data_weather["Max Temp"] <= 80)].dropna()
narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
125,Villarrica,0,PY,1603268408,69,-25.75,-56.43,71.44,1.74
152,Shahreza,0,IR,1603268414,15,32.03,51.88,71.13,4.90
206,Buraidah,0,SA,1603268430,26,26.33,43.98,78.80,4.70
213,Kruisfontein,0,ZA,1603268203,44,-34.00,24.73,70.00,1.99
331,Torbat-e Jām,0,IR,1603268469,16,35.24,60.62,74.57,9.15
414,Yangi Marg`ilon,0,UZ,1603268493,26,40.43,71.72,71.60,4.70
539,Herat Province,0,AF,1603268527,11,34.50,62.00,73.40,3.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
125,Villarrica,PY,-25.75,-56.43,
152,Shahreza,IR,32.03,51.88,
206,Buraidah,SA,26.33,43.98,
213,Kruisfontein,ZA,-34.00,24.73,
331,Torbat-e Jām,IR,35.24,60.62,
414,Yangi Marg`ilon,UZ,40.43,71.72,
539,Herat Province,AF,34.50,62.00,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
              "keyword" : "hotel",
              "radius" : 5000,
              "key" : g_key}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f"{lat},{lng}"
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("-"*60)
    time.sleep(1)

print("END")

Retrieving Results for Index 125: Villarrica.
Closest hotel in Villarrica is Villarrica Palace Hotel.
------------------------------------------------------------
Retrieving Results for Index 152: Shahreza.
Closest hotel in Shahreza is هتل باغ اناری.
------------------------------------------------------------
Retrieving Results for Index 206: Buraidah.
Closest hotel in Buraidah is فندق براند القصيم.
------------------------------------------------------------
Retrieving Results for Index 213: Kruisfontein.
Missing field/result... skipping.
------------------------------------------------------------
Retrieving Results for Index 331: Torbat-e Jām.
Closest hotel in Torbat-e Jām is Pardisan Hotel 🏩.
------------------------------------------------------------
Retrieving Results for Index 414: Yangi Marg`ilon.
Closest hotel in Yangi Marg`ilon is Diyor hotel.
------------------------------------------------------------
Retrieving Results for Index 539: Herat Province.
Missing field/result.

In [8]:
# Updated dataframe with Hotel Names
hotel_df

,City,Country,Lat,Lng,Hotel Name
125,Villarrica,PY,-25.75,-56.43,Villarrica Palace Hotel
152,Shahreza,IR,32.03,51.88,هتل باغ اناری
206,Buraidah,SA,26.33,43.98,فندق براند القصيم
213,Kruisfontein,ZA,-34.00,24.73,
331,Torbat-e Jām,IR,35.24,60.62,Pardisan Hotel 🏩
414,Yangi Marg`ilon,UZ,40.43,71.72,Diyor hotel
539,Herat Province,AF,34.50,62.00,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))